In [1]:
!pip3 install -q torchaudio torchvision mne loguru neo

In [2]:
import sys
sys.path.append('/Users/theomaetz/Downloads/MA3/INL/Setup/lighthouse-features-preprocessing/')

In [3]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchaudio
import torchvision
import torchvision.transforms as T

import lighthouse.data_loader.files_folders as ff
import lighthouse.metadata as metadata
from lighthouse.data_loader.load_chuv import load_absd, load_dt5, load_gdp, load_smr
from lighthouse.data_loader.torch_dataset import TimeseriesDataset
from lighthouse.preprocessing.transform import MNEFilter

In [4]:
os.environ["DATA_DIR"] = "/Volumes"
training_sessions = metadata.get_training_sessions()
absd_dir = ff.find_absd_dirs(training_sessions[0])[0]
print(absd_dir)

2024-11-06 01:11:07.844 | WARNING  | lighthouse.metadata:get_training_sessions:125 - Only found 65 out of 66 training sessions


/Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data


In [5]:
SFREQ = 585
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [ ]:
def load_absd_ecog(absd_dir):
    signals, ch_names = load_absd(absd_dir)
    ecog_ch_names = [ch for ch in ch_names if "ecog" in ch]
    ecog_signals = signals[[ch_names.index(ch) for ch in ecog_ch_names]]
    return ecog_signals


def load_absd_states(absd_dir, return_ch_names=False):
    signals, ch_names = load_absd(absd_dir, return_states=True)
    state_ch_names = [ch for ch in ch_names if "state__" in ch]
    state_signals = signals[[ch_names.index(ch) for ch in state_ch_names]]
    if return_ch_names:
        return state_signals, state_ch_names
    return state_signals

In [25]:
state_signals, state_ch_names = load_absd_states(absd_dir, return_ch_names=True)

2024-11-06 00:59:00.500 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_01of02.dt5
2024-11-06 01:00:06.736 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-11-06 01:00:06.778 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-11-06 01:00:15.489 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_02of02.dt5


In [26]:
state_ch_names

['state__idle',
 'state__shoulder__flexion',
 'state__shoulder__extension',
 'state__shoulder__adduction',
 'state__shoulder__abduction',
 'state__shoulder__internal_rotation',
 'state__shoulder__external_rotation',
 'state__elbow__flexion',
 'state__elbow__extension',
 'state__wrist__pronation',
 'state__wrist__supination',
 'state__hand__open',
 'state__hand__close']

In [27]:
x_data = load_absd_ecog(absd_dir)

2024-11-06 01:01:13.240 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_01of02.dt5
2024-11-06 01:02:14.906 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-11-06 01:02:14.917 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-11-06 01:02:20.293 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_02of02.dt5


In [28]:
x_data.shape

(32, 353115)

In [ ]:
state_signals.shape

(13, 353115)

: 

In [ ]:
win_size = 585
win_step = 59
n_fft = win_size
hop_length = 10
dataset = TimeseriesDataset(
    load_x_func=load_absd_ecog,
    load_x_args={"absd_dir": absd_dir},
    load_y_func=load_absd_states,
    load_y_args={"absd_dir": absd_dir},
    n_samples_step=win_step,
    n_samples_win=win_size,
    x_preprocess=T.Compose(
        [
            MNEFilter(sfreq=SFREQ, l_freq=1, h_freq=200, notch_freqs=np.arange(50, 201, 50), apply_car=True),
            torchvision.transforms.ToTensor(),
        ],
    ),
    y_preprocess=T.Compose(
        [],
    ),
    x_transform=T.Compose( 
        [
            # we first push to GPU, then apply transforms
            lambda x: x.to(DEVICE),
            # needs to be adjusted to the desired output size (FREQ_BINS, TIME_BINS)
            torchaudio.transforms.Spectrogram(
                n_fft=n_fft,
                win_length=n_fft,
                hop_length=hop_length,
                center=True,
                window_fn=lambda x: torch.hann_window(x).to(DEVICE),
            ),
            lambda x: x.squeeze(0).float(),
        ]
    ),
    y_transform=T.Compose(
        [
            lambda x: torch.tensor(x).to(DEVICE),
            # for the sake of this example, we take the last state value
            lambda x: x[..., -1].flatten().float(),
        ]
    ),
    precompute=True,
)

2024-11-06 01:11:11.927 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_01of02.dt5
2024-11-06 01:11:30.173 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-11-06 01:11:30.176 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-11-06 01:11:34.926 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_02of02.dt5
2024-11-06 01:11:56.894 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_0

Creating RawArray with float64 data, n_channels=32, n_times=353115
    Range : 0 ... 353114 =      0.000 ...   603.614 secs
Ready.
ECoG channel type selected for re-referencing
Applying average reference.
Applying a custom ('ECoG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3861 samples (6.600 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 200.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 225.00 Hz)
- Filter length: 1931 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.3s finished
2024-11-06 01:12:44.821 | INFO     | lighthouse.data_loader.torch_dataset:__init__:104 - Precomputing dataset (could take a while)


...

After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])


100%|█████████▉| 5966/5976 [01:41<00:00, 40.18it/s]

After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])


100%|██████████| 5976/5976 [01:42<00:00, 58.52it/s]


After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])
Before transform: torch.Size([1, 32, 585])
After transform: torch.Size([32, 293, 59])


In [8]:
print(type(dataset))

<class 'lighthouse.data_loader.torch_dataset.TimeseriesDataset'>


In [9]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False)

In [10]:
X, y = next(iter(dataloader))

In [11]:
X.shape, y.shape

(torch.Size([8, 32, 293, 59]), torch.Size([8, 13]))

In [12]:
y[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])